# Limpieza del csv champions

In [4520]:
import pandas as pd
import re

archivo_csv = '../data/datos_champions.csv'

# Cargar el archivo CSV
df = pd.read_csv(archivo_csv)

# Eliminar filas completamente vacías
df.dropna(how='all', inplace=True)

# Función para quitar prefijos basado en una lista de códigos de países conocidos
def quitar_prefijos(nombre, prefijos):
    if pd.isnull(nombre):
        return nombre  # Retorna el valor NaN sin cambios
    # Quitar prefijo al inicio
    for prefijo in prefijos:
        if nombre.lower().startswith(prefijo):
            nombre = re.sub(f'^{prefijo}', '', nombre, flags=re.IGNORECASE).strip()
            break  # Si se encuentra y quita un prefijo, no se necesita seguir buscando
    # Quitar prefijo al final 
    for prefijo in prefijos:
        if nombre.lower().endswith(prefijo):
            nombre = re.sub(f'{prefijo}$', '', nombre, flags=re.IGNORECASE).strip()
            break  # Si se encuentra y quita un prefijo, no se necesita seguir buscando
    return nombre

# Lista de prefijos conocidos que deseas eliminar (en minúsculas)
prefijos = ['eng', 'es', 'fr', 'it', 'de', 'pt', 'ru', 'cz', 'nl', 'gr', 'sc', 'sw', 'tr', 'uk', 'be', 'pl', 'ro', 'ie', 'al', 'at', 'bg', 'ch', 
            'cy', 'dk', 'fi', 'hr', 'hu', 'is', 'lt', 'lv', 'mk', 'mt', 'no', 'rs', 'se', 'si', 'sk', 'ua', 'ar', 'bo', 'br', 'cl', 'co', 
            'ec', 'mx', 'py', 'pe', 'uy', 've', 'ca', 'us', 'cn', 'jp', 'kr', 'au', 'nz', 'in', 'id', 'my', 'ph', 'sg', 'th', 'vn', 'ae',
              'sa', 'il', 'eg', 'ma', 'za', 'dz', 'tn', 'ma', 'sn', 'cm', 'ci', 'gh', 'ng', 'ke', 'ug', 'za', 'et', 'sd', 'so', 'ly', 'mr',
                'gm', 'gw', 'lr', 'sl', 'bf', 'ne', 'tg', 'bj', 'cv', 'st', 'gq', 'ga', 'cg', 'cd', 'cf', 'td', 'cm']

# Aplicar la función para quitar prefijos en las columnas 'Home' y 'Away'
df['Home'] = df['Home'].apply(lambda x: quitar_prefijos(x, prefijos))
df['Away'] = df['Away'].apply(lambda x: quitar_prefijos(x, prefijos))



In [4521]:
#Borramos ahora aquellas filas que el unico dato que se tienen es de 'season'

filas_a_eliminar = df[df.isnull().sum(axis=1) >= (len(df.columns) - 1)].index

# Eliminar esas filas
df.drop(filas_a_eliminar, inplace=True)


In [4522]:

# La primera fila es el encabezado, así que la guardamos para comparaciones
encabezado = df.columns.tolist()

# Función para detectar filas que son como el encabezado pero con un año en la primera columna
def es_fila_encabezado_con_anio(fila):
    # Comparamos cada elemento de la fila con el encabezado, excepto el primero
    return all(fila[i] == encabezado[i] for i in range(1, len(encabezado)))

# Identificamos las filas que cumplen con esta condición
filas_a_eliminar = [i for i, fila in df.iterrows() if es_fila_encabezado_con_anio(fila)]

# Eliminamos estas filas del DataFrame
df.drop(index=filas_a_eliminar, inplace=True)

# Guardamos el DataFrame limpio, manteniendo el encabezado
df.to_csv('../data/datos_champions_limpio.csv', index=False)



In [4523]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 574 entries, 0 to 652
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Season        574 non-null    object
 1   Round         574 non-null    object
 2   Day           574 non-null    object
 3   Date          574 non-null    object
 4   Time          255 non-null    object
 5   Home          574 non-null    object
 6   Score         574 non-null    object
 7   Away          574 non-null    object
 8   Attendance    214 non-null    object
 9   Venue         574 non-null    object
 10  Referee       574 non-null    object
 11  Match Report  574 non-null    object
 12  Notes         553 non-null    object
dtypes: object(13)
memory usage: 62.8+ KB


In [4524]:
print(df.columns)

Index(['Season', 'Round', 'Day', 'Date', 'Time', 'Home', 'Score', 'Away',
       'Attendance', 'Venue', 'Referee', 'Match Report', 'Notes'],
      dtype='object')


In [4525]:
# Borramos las columnas que no necesitamos para el análisis
columns_to_drop = ['Time', 'Attendance', 'Notes', 'Match Report', 'Referee', 'Venue']

# Eliminar las columnas
df.drop(columns=columns_to_drop, inplace=True)

# Verificar los datos después de la limpieza
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 574 entries, 0 to 652
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Season  574 non-null    object
 1   Round   574 non-null    object
 2   Day     574 non-null    object
 3   Date    574 non-null    object
 4   Home    574 non-null    object
 5   Score   574 non-null    object
 6   Away    574 non-null    object
dtypes: object(7)
memory usage: 35.9+ KB


Como vemos tenemos datos de tipo objeto así que lo que haremos es convertir estos a enteros para poder trabajar con ellos en modelos de aprendizaje

In [4526]:
print(df['Score'].value_counts())


Score
1–0          66
1–1          51
0–0          44
2–1          44
2–0          40
0–1          39
0–2          32
3–1          30
1–2          29
3–0          24
2–2          21
1–3          18
2–3          15
4–0          15
0–3          14
3–2          13
4–1          13
4–2           7
1–4           5
7–0           4
5–1           4
7–1           4
0–4           3
0–5           3
5–2           3
4–3           2
2–4           2
6–1           2
2–5           2
3–4           2
3–3           2
5–0           2
4–4           1
2–8           1
7–2           1
(4)1–1(2)     1
1–5           1
5–3           1
(5)1–1(3)     1
(8)0–0(7)     1
(2)3–3(3)     1
(3)1–0(2)     1
(4)1–0(1)     1
1–6           1
(2)3–2(3)     1
(1)1–0(4)     1
(3)1–1(4)     1
(1)2–1(3)     1
(6)1–0(7)     1
(4)1–0(3)     1
(6)1–1(5)     1
Name: count, dtype: int64


Parece que la mayoría de las entradas siguen el formato esperado de "home_goals-away_goals". Sin embargo, hay algunas entradas que tienen un formato diferente, como "(6)1-0(7)" o "(4)1-0(3)".

In [4527]:
import pandas as pd
import re

# Simulando la carga de tu DataFrame (reemplaza esto con la carga de tu archivo CSV)
# df = pd.read_csv('tu_archivo.csv')

# Convertir la columna 'Date' a datetime
df['Date'] = pd.to_datetime(df['Date'])

# Codificar la columna 'Round' como variables dummy
round_dummies = pd.get_dummies(df['Round'], prefix='Round')
df = pd.concat([df, round_dummies], axis=1)

# Función ajustada para extraer goles
def extract_goals(text):
    # Primero, intentamos extraer resultados que incluyen penales
    match_penalties = re.match(r".*\((\d+)\).*–.*\((\d+)\)", text)
    if match_penalties:
        # Si hay resultados de penales, estos son los resultados "definitivos"
        return int(match_penalties.group(1)), int(match_penalties.group(2))
    
    # Si no hay penales, buscamos el resultado del tiempo regular
    match_regular = re.match(r"(\d+)–(\d+)", text)
    if match_regular:
        return int(match_regular.group(1)), int(match_regular.group(2))
    
    # Si no se encuentra ningún formato conocido, retornamos None para indicar datos faltantes
    return None, None

# Aplicar la función ajustada
df['HomeGoals'], df['AwayGoals'] = zip(*df['Score'].map(extract_goals))

# Eliminar las columnas originales que ya no se necesitan
df.drop(['Round', 'Score'], axis=1, inplace=True)

# Nota: En este punto, no rellenaremos los NaN con ceros como antes.

# Solo convertimos a enteros las filas que no son NaN
# Esto requiere manejar los NaN de manera diferente, por ejemplo, manteniéndolos como NaN o usando un marcador especial para los datos faltantes.

# Muestra la información del DataFrame para verificar los cambios
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 574 entries, 0 to 652
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Season                574 non-null    object        
 1   Day                   574 non-null    object        
 2   Date                  574 non-null    datetime64[ns]
 3   Home                  574 non-null    object        
 4   Away                  574 non-null    object        
 5   Round_Final           574 non-null    bool          
 6   Round_Quarter-finals  574 non-null    bool          
 7   Round_Round of 16     574 non-null    bool          
 8   Round_Semi-finals     574 non-null    bool          
 9   HomeGoals             574 non-null    int64         
 10  AwayGoals             574 non-null    int64         
dtypes: bool(4), datetime64[ns](1), int64(2), object(4)
memory usage: 38.1+ KB


# Limpieza csv overall


Una vez que tenemos limpio el de los datos de la champions pasamos a limpiar el del overall


Al igual que en el anterior csv nos encontramos con que los nombres tienen prefijo luego es lo primero que limpiaremos y solucionaremos

In [4528]:
archivo2_csv = '../data/overall.csv'

# Cargar el archivo CSV
df2 = pd.read_csv(archivo2_csv)

In [4529]:
print(df2.shape)
df2.head()

(759, 15)


,Season,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Attendance,Top Team Scorer,Goalkeeper,Notes
0,2022-2023,1,engManchester City,13,8,5,0,32,5,+27,29,"63,639",Erling Haaland-12,Ederson,NaN
1,2022-2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-2023,2,itInter,13,7,3,3,19,11,+8,24,"71,415",Edin Džeko-4,André Onana,NaN
3,2022-2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-2023,SF,esReal Madrid,12,8,2,2,26,13,+13,26,"58,761",Vinicius Júnior-7,Thibaut Courtois,NaN


Observamos que hay filas vacías, luego estas hay que borrarlas

In [4530]:
#Borramos ahora aquellas filas que el unico dato que se tienen es de 'season'

filas_a_eliminar = df2[df2.isnull().sum(axis=1) >= (len(df2.columns) - 1)].index

# Eliminar esas filas
df2.drop(filas_a_eliminar, inplace=True)


In [4531]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 659 entries, 0 to 758
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Season           659 non-null    object
 1   Rk               659 non-null    object
 2   Squad            659 non-null    object
 3   MP               659 non-null    object
 4   W                659 non-null    object
 5   D                659 non-null    object
 6   L                659 non-null    object
 7   GF               659 non-null    object
 8   GA               659 non-null    object
 9   GD               659 non-null    object
 10  Pts              659 non-null    object
 11  Attendance       644 non-null    object
 12  Top Team Scorer  656 non-null    object
 13  Goalkeeper       659 non-null    object
 14  Notes            9 non-null      object
dtypes: object(15)
memory usage: 82.4+ KB


Como observamos, la última columna contiene muy pocos datos los cuales tampoco son relevantes por lo que procederemos a borrarla

In [4532]:
df2.drop(["Notes"],axis=1,inplace=True)

In [4533]:
#Como vemos entonces la columna 'Notes' ya no se encuentra en el DataFrame
df2.columns

Index(['Season', 'Rk', 'Squad', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts',
       'Attendance', 'Top Team Scorer', 'Goalkeeper'],
      dtype='object')

In [4534]:
#Cuando vemos el CSV nos damos cuenta que el nombre del jugador tiene un guion y luego los goles que metio en la temporada

#Vamos a separar el nombre del jugador y los goles que metio en la temporada
df2[['Top Team Scorer', 'Top Team Scorer Goals']] = df2['Top Team Scorer'].str.rsplit('-', n=1, expand=True)

# Nos aseguramos de que se hayan hecho los cambios correctamente
df2.head()

,Season,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Attendance,Top Team Scorer,Goalkeeper,Top Team Scorer Goals
0,2022-2023,1,engManchester City,13,8,5,0,32,5,+27,29,"63,639",Erling Haaland,Ederson,12
2,2022-2023,2,itInter,13,7,3,3,19,11,+8,24,"71,415",Edin Džeko,André Onana,4
4,2022-2023,SF,esReal Madrid,12,8,2,2,26,13,+13,26,"58,761",Vinicius Júnior,Thibaut Courtois,7
5,2022-2023,SF,itMilan,12,5,3,4,15,11,+4,18,"72,546",Olivier Giroud,Mike Maignan,5
7,2022-2023,QF,deBayern Munich,10,8,1,1,22,6,+16,25,"75,000","Leroy Sané,Eric Maxim Choupo-Moting",Yann Sommer,4


Como vemos se ha separado correctamente

In [4535]:
df2.shape

(659, 15)

Como vemos el CSV tras los cambios realizados se ha quedado con 659 filas y 15 columnas que son las siguientes:

* 'Season': Temporada del equipo.
* 'Rk': Clasificación del equipo en la temporada.
* 'Squad': Nombre del equipo.
* 'MP': Partidos jugados.
* 'W': Partidos ganados.
* 'D': Partidos empatados.
* 'L': Partidos perdidos.
* 'GF': Goles a favor.
* 'GA': Goles en contra.
* 'GD': Diferencia de goles (GF - GA).
* 'Pts': Puntos obtenidos.
* 'Attendance': Asistencia promedio en los partidos.
* 'Top Team Scorer': Máximo goleador del equipo.
* 'Goalkeeper': Portero del equipo.
* 'Top Team Scorer Goals': Cantidad de goles anotados por el máximo goleador del equipo.


In [4536]:
df2.head()

,Season,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Attendance,Top Team Scorer,Goalkeeper,Top Team Scorer Goals
0,2022-2023,1,engManchester City,13,8,5,0,32,5,+27,29,"63,639",Erling Haaland,Ederson,12
2,2022-2023,2,itInter,13,7,3,3,19,11,+8,24,"71,415",Edin Džeko,André Onana,4
4,2022-2023,SF,esReal Madrid,12,8,2,2,26,13,+13,26,"58,761",Vinicius Júnior,Thibaut Courtois,7
5,2022-2023,SF,itMilan,12,5,3,4,15,11,+4,18,"72,546",Olivier Giroud,Mike Maignan,5
7,2022-2023,QF,deBayern Munich,10,8,1,1,22,6,+16,25,"75,000","Leroy Sané,Eric Maxim Choupo-Moting",Yann Sommer,4


In [4537]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 659 entries, 0 to 758
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Season                 659 non-null    object
 1   Rk                     659 non-null    object
 2   Squad                  659 non-null    object
 3   MP                     659 non-null    object
 4   W                      659 non-null    object
 5   D                      659 non-null    object
 6   L                      659 non-null    object
 7   GF                     659 non-null    object
 8   GA                     659 non-null    object
 9   GD                     659 non-null    object
 10  Pts                    659 non-null    object
 11  Attendance             644 non-null    object
 12  Top Team Scorer        656 non-null    object
 13  Goalkeeper             659 non-null    object
 14  Top Team Scorer Goals  637 non-null    object
dtypes: object(15)
memory usage: 

Cuando imprimimos la información de las columnas vemos que se nos muestra todos como tipo objeto, cosa que algunas columnas no es cierta por lo que vamos a convertirlo.

In [4538]:
columnas_numericas = ['MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts', 'Top Team Scorer Goals']

# Convertimos las columnas numéricas al tipo de datos apropiado
for columna in columnas_numericas:
    df2[columna] = pd.to_numeric(df2[columna], errors='coerce')  # 'coerce' convierte los valores no numéricos a NaN

# Verificamos que las columnas se hayan convertido correctamente
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
Index: 659 entries, 0 to 758
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Season                 659 non-null    object 
 1   Rk                     659 non-null    object 
 2   Squad                  659 non-null    object 
 3   MP                     640 non-null    float64
 4   W                      640 non-null    float64
 5   D                      640 non-null    float64
 6   L                      640 non-null    float64
 7   GF                     640 non-null    float64
 8   GA                     640 non-null    float64
 9   GD                     640 non-null    float64
 10  Pts                    640 non-null    float64
 11  Attendance             644 non-null    object 
 12  Top Team Scorer        656 non-null    object 
 13  Goalkeeper             659 non-null    object 
 14  Top Team Scorer Goals  637 non-null    float64
dtypes: float64(

In [4539]:
# Remover comas y otros caracteres no numéricos de la columna "Attendance"
df2['Attendance'] = df2['Attendance'].str.replace('[^\d.]', '', regex=True)

# Convertir la columna "Attendance" a tipo flotante y manejar los errores
df2['Attendance'] = pd.to_numeric(df2['Attendance'], errors='coerce')

# Llenar los valores nulos (NaN) con cero
df2['Attendance'].fillna(0, inplace=True)

# Convertir la columna "Attendance" a tipo flotante
df2['Attendance'] = df2['Attendance'].astype(float)

df2.info()


<class 'pandas.core.frame.DataFrame'>
Index: 659 entries, 0 to 758
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Season                 659 non-null    object 
 1   Rk                     659 non-null    object 
 2   Squad                  659 non-null    object 
 3   MP                     640 non-null    float64
 4   W                      640 non-null    float64
 5   D                      640 non-null    float64
 6   L                      640 non-null    float64
 7   GF                     640 non-null    float64
 8   GA                     640 non-null    float64
 9   GD                     640 non-null    float64
 10  Pts                    640 non-null    float64
 11  Attendance             659 non-null    float64
 12  Top Team Scorer        656 non-null    object 
 13  Goalkeeper             659 non-null    object 
 14  Top Team Scorer Goals  637 non-null    float64
dtypes: float64(

In [4540]:
# Con la cariable thresh le decimos que queremos que elimine las filas que tengan al menos dos valores nulos
# También reseteamos el índice
df2 = df2.dropna(thresh=2).reset_index(drop=True)

# Nos aseguramos de que se hayan eliminado correctamente
print(df2.shape)
df2.head()

(659, 15)


,Season,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Attendance,Top Team Scorer,Goalkeeper,Top Team Scorer Goals
0,2022-2023,1,engManchester City,13.0,8.0,5.0,0.0,32.0,5.0,27.0,29.0,63639.0,Erling Haaland,Ederson,12.0
1,2022-2023,2,itInter,13.0,7.0,3.0,3.0,19.0,11.0,8.0,24.0,71415.0,Edin Džeko,André Onana,4.0
2,2022-2023,SF,esReal Madrid,12.0,8.0,2.0,2.0,26.0,13.0,13.0,26.0,58761.0,Vinicius Júnior,Thibaut Courtois,7.0
3,2022-2023,SF,itMilan,12.0,5.0,3.0,4.0,15.0,11.0,4.0,18.0,72546.0,Olivier Giroud,Mike Maignan,5.0
4,2022-2023,QF,deBayern Munich,10.0,8.0,1.0,1.0,22.0,6.0,16.0,25.0,75000.0,"Leroy Sané,Eric Maxim Choupo-Moting",Yann Sommer,4.0


Vemos que ahora si tienen todas el formato que deben de tener

Ahora vamos a pasar a la limpieza del nombre del equipo el cual contiene el prefijo del país delante, para ello usaremos una libreria y guardaremos en una nueva columna el pais.

In [4541]:
paises_dict = {
    'Germany': 'de',
    'Spain': 'es',
    'England': 'eng',
    'Italy': 'it',
    'France': 'fr',
    'Netherlands': 'nl',
    'Portugal': 'pt',
    'Belgium': 'be',
    'Scotland': 'sct',
    'Ireland': 'ie',
    'Wales': 'wal',
    'Czech Republic': 'cz',
    'Poland': 'pl',
    'Switzerland': 'ch',
    'Austria': 'at',
    'Denmark': 'dk',
    'Sweden': 'se',
    'Norway': 'no',
    'Slovenia': 'si',
    'Slovakia': 'sk',
    'Hungary': 'hu',
    'Romania': 'ro',
    'Bulgaria': 'bg',
    'Croatia': 'hr',
    'Serbia': 'rs',
    'Greece': 'gr',
    'Moldova': 'md',
    'Cyprus': 'cy',
    'Russia': 'ru',
    'Belarus': 'by',
    'Ukraine': 'ua',
    'Turkey': 'tr',
    'Israel': 'il',
    'Azerbaijan': 'az',
    'Kazakhstan': 'kz'
}


In [4542]:
# Lista para almacenar los países
paises = []

# Iterar sobre el DataFrame df2
for i, fila in enumerate(df2['Squad']):
    # Bandera para verificar si se encontró una coincidencia en el diccionario
    coincidencia = False
    
    # Iterar sobre el diccionario de países
    for key, value in paises_dict.items():
        # Verificar si el nombre del equipo comienza o termina con el valor del diccionario
        if isinstance(fila, str) and (fila.startswith(value) or fila.endswith(value)):
            # Reemplazar el valor con la cadena vacía
            df2.loc[i, 'Squad'] = fila.replace(value, '')
            # Agregar el país a la lista
            paises.append(key)
            coincidencia = True
            break 
    
    # Si no se encontró una coincidencia en el diccionario, agregar una cadena vacía al país
    if not coincidencia:
        # Agregar un país predeterminado
        paises.append('Unknown')

# Agregar la columna 'Country' al DataFrame
df2['Country'] = paises

# Verificar si se ha agregado correctamente
df2.head()

   

,Season,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Attendance,Top Team Scorer,Goalkeeper,Top Team Scorer Goals,Country
0,2022-2023,1,Manchester City,13.0,8.0,5.0,0.0,32.0,5.0,27.0,29.0,63639.0,Erling Haaland,Ederson,12.0,England
1,2022-2023,2,Inter,13.0,7.0,3.0,3.0,19.0,11.0,8.0,24.0,71415.0,Edin Džeko,André Onana,4.0,Italy
2,2022-2023,SF,Real Madrid,12.0,8.0,2.0,2.0,26.0,13.0,13.0,26.0,58761.0,Vinicius Júnior,Thibaut Courtois,7.0,Spain
3,2022-2023,SF,Milan,12.0,5.0,3.0,4.0,15.0,11.0,4.0,18.0,72546.0,Olivier Giroud,Mike Maignan,5.0,Italy
4,2022-2023,QF,Bayern Munich,10.0,8.0,1.0,1.0,22.0,6.0,16.0,25.0,75000.0,"Leroy Sané,Eric Maxim Choupo-Moting",Yann Sommer,4.0,Germany


In [4543]:
#Ahora vamos a ver los valores que puede tomar la columa 'Rk' que hace referencia a la posición en la champions league
df2['Rk'].value_counts()


Rk
GR       320
R16      160
QF        80
SF        40
Squad     19
W         18
F         18
1          2
2          2
Name: count, dtype: int64

Como vemos aparece el 1 y el 2 como valores que la columna 'Rk' puede tomar estos correponde a 'W' y 'F' respectivamente así que transformemoslo.

In [4544]:
#Transformamos los valores '1' y '2' de la columna 'Rk' a 'W' y 'F' respectivamente
# Reemplazamos los valores de la columna 'Rk' por 'W' y 'F' 
df2['Rk'] = df2['Rk'].replace('1', 'W')
df2['Rk'] = df2['Rk'].replace('2', 'F')

df2['Rk'].value_counts()


Rk
GR       320
R16      160
QF        80
SF        40
W         20
F         20
Squad     19
Name: count, dtype: int64

In [4545]:
# Filtrar filas donde la columna Rk tiene el valor "Squad"
squad_rows = df2[df2['Rk'] == 'Squad']
print(squad_rows)




    Season     Rk Squad  MP   W   D   L  GF  GA  GD  Pts  Attendance  \
32      Rk  Squad    MP NaN NaN NaN NaN NaN NaN NaN  NaN         0.0   
65      Rk  Squad    MP NaN NaN NaN NaN NaN NaN NaN  NaN         0.0   
98      Rk  Squad    MP NaN NaN NaN NaN NaN NaN NaN  NaN         0.0   
131     Rk  Squad    MP NaN NaN NaN NaN NaN NaN NaN  NaN         0.0   
164     Rk  Squad    MP NaN NaN NaN NaN NaN NaN NaN  NaN         0.0   
197     Rk  Squad    MP NaN NaN NaN NaN NaN NaN NaN  NaN         0.0   
230     Rk  Squad    MP NaN NaN NaN NaN NaN NaN NaN  NaN         0.0   
263     Rk  Squad    MP NaN NaN NaN NaN NaN NaN NaN  NaN         0.0   
296     Rk  Squad    MP NaN NaN NaN NaN NaN NaN NaN  NaN         0.0   
329     Rk  Squad    MP NaN NaN NaN NaN NaN NaN NaN  NaN         0.0   
362     Rk  Squad    MP NaN NaN NaN NaN NaN NaN NaN  NaN         0.0   
395     Rk  Squad    MP NaN NaN NaN NaN NaN NaN NaN  NaN         0.0   
428     Rk  Squad    MP NaN NaN NaN NaN NaN NaN NaN  NaN        

In [4546]:
# Filtrar filas donde la columna 'Rk' tiene el valor 'Squad'
squad_rows_index = df2[df2['Rk'] == 'Squad'].index

# Eliminar las filas que contienen 'Squad' en la columna 'Rk'
overall_cleaned = df2.drop(index=squad_rows_index)

# Contar los valores en la columna 'Rk' en el DataFrame limpio
rankings = overall_cleaned['Rk'].value_counts()

# Imprimir los valores contados
print(rankings)



Rk
GR     320
R16    160
QF      80
SF      40
W       20
F       20
Name: count, dtype: int64


In [4547]:
#lo pasamos tod a csv que sea csv limpio para poder trabajar con el
overall_cleaned.to_csv('../data/overall_limpio.csv', index=False)

# Limpiamos el CSV de jugadores

In [4548]:
df3=pd.read_csv('../data/jugadores.csv')
df3.shape

(5221, 38)

In [4549]:
df3.head()  

,Temporada,#,Jugador,País,Posc,Equipo,Edad,Nacimiento,PJ,Titular,...,Ast90,G+A90,G-TP90,G+A-TP90,xG90,xAG90,xG+xAG90,npxG90,npxG+xAG90,Partidos
0,2017-2018,1,Vincent Aboubakar,cmCMR,DL,ptPorto,25,1992,6,6,...,0.36,1.27,0.91,1.27,0.51,0.19,0.70,0.51,0.70,Partidos
1,2017-2018,2,Marcos Acuña,arARG,"DL,DF",ptSporting CP,25,1991,5,5,...,0.21,0.21,0.00,0.21,0.00,0.11,0.11,0.00,0.11,Partidos
2,2017-2018,3,Tosin Adarabioyo,engENG,DF,engManchester City,19,1997,2,1,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Partidos
3,2017-2018,4,Adriano,brBRA,DF,trBeşiktaş,32,1984,6,5,...,0.00,0.00,0.00,0.00,0.01,0.02,0.03,0.01,0.03,Partidos
4,2017-2018,5,Luiz Adriano,brBRA,"DL,CC",ruSpartak Moscow,30,1987,6,6,...,0.00,0.17,0.17,0.17,0.16,0.15,0.32,0.16,0.32,Partidos


In [4550]:
df3.columns

Index(['Temporada', '#', 'Jugador', 'País', 'Posc', 'Equipo', 'Edad',
       'Nacimiento', 'PJ', 'Titular', 'Mín', '90 s', 'Gls.', 'Ass', 'G+A',
       'G-TP', 'TP', 'TPint', 'TA', 'TR', 'xG', 'npxG', 'xAG', 'npxG+xAG',
       'PrgC', 'PrgP', 'PrgR', 'Gls90.', 'Ast90', 'G+A90', 'G-TP90',
       'G+A-TP90', 'xG90', 'xAG90', 'xG+xAG90', 'npxG90', 'npxG+xAG90',
       'Partidos'],
      dtype='object')

Vemos que tenemos una tabla con 5221 filas y 38 columnas que son:


- 'Temporada': Representa la temporada en la que se jugó el partido.
- '#': Representa el número de jugador.
- 'Jugador': Representa el nombre del jugador.
- 'País': Representa el país del jugador.
- 'Posc': Representa la posición del jugador.
- 'Equipo': Representa el equipo al que pertenece el jugador.
- 'Edad': Representa la edad del jugador.
- 'Nacimiento': Representa la fecha de nacimiento del jugador.
- 'PJ': Representa el número de partidos jugados por el jugador.
- 'Titular': Indica si el jugador fue titular en el partido.
- 'Mín 90 s': Representa los minutos jugados por el jugador.
- 'Gls.': Representa el número de goles marcados por el jugador.
- 'Ass': Representa el número de asistencias realizadas por el jugador.
- 'G+A': Representa la suma de goles y asistencias del jugador.
- 'G-TP': Representa el número de goles marcados desde el punto de penalti por el jugador.
- 'TP': Representa el número de penaltis lanzados por el jugador.
- 'TPint': Representa el número de penaltis convertidos por el jugador.
- 'TA': Representa el número de tarjetas amarillas recibidas por el jugador.
- 'TR': Representa el número de tarjetas rojas recibidas por el jugador.
- 'xG': Representa el valor esperado de goles del jugador.
- 'npxG': Representa el valor esperado de goles sin contar los goles de penalti del jugador.
- 'xAG': Representa el valor esperado de goles en contra del jugador.
- 'npxG+xAG': Representa la suma del valor esperado de goles sin contar los goles de penalti y el valor esperado de goles en contra del jugador.
- 'PrgC': Representa el número de pases completados por el jugador.
- 'PrgP': Representa el número de pases intentados por el jugador.
- 'PrgR': Representa el porcentaje de pases completados por el jugador.
- 'Gls90.': Representa el número de goles por cada 90 minutos jugados por el jugador.
- 'Ast90': Representa el número de asistencias por cada 90 minutos jugados por el jugador.
- 'G+A90': Representa la suma de goles y asistencias por cada 90 minutos jugados por el jugador.
- 'G-TP90': Representa el número de goles marcados desde el punto de penalti por cada 90 minutos jugados por el jugador.
- 'G+A-TP90': Representa la suma de goles y asistencias sin contar los goles de penalti por cada 90 minutos jugados por el jugador.
- 'xG90': Representa el valor esperado de goles por cada 90 minutos jugados por el jugador.
- 'xAG90': Representa el valor esperado de goles en contra por cada 90 minutos jugados por el jugador.
- 'xG+xAG90': Representa la suma del valor esperado de goles y el valor esperado de goles en contra por cada 90 minutos jugados por el jugador.
- 'npxG90': Representa el valor esperado de goles sin contar los goles de penalti por cada 90 minutos jugados por el jugador.
- 'npxG+xAG90': Representa la suma del valor esperado de goles sin contar los goles de penalti y el valor esperado de goles en contra por cada 90 minutos jugados por el jugador.
- 'Partidos': Representa el número de partidos jugados por el jugador.



In [4551]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5221 entries, 0 to 5220
Data columns (total 38 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Temporada   5221 non-null   object
 1   #           5221 non-null   object
 2   Jugador     5221 non-null   object
 3   País        5216 non-null   object
 4   Posc        5220 non-null   object
 5   Equipo      5221 non-null   object
 6   Edad        5220 non-null   object
 7   Nacimiento  5220 non-null   object
 8   PJ          5221 non-null   object
 9   Titular     5221 non-null   object
 10  Mín         5220 non-null   object
 11  90 s        5221 non-null   object
 12  Gls.        5221 non-null   object
 13  Ass         5221 non-null   object
 14  G+A         5221 non-null   object
 15  G-TP        5221 non-null   object
 16  TP          5221 non-null   object
 17  TPint       5221 non-null   object
 18  TA          5221 non-null   object
 19  TR          5221 non-null   object
 20  xG      

In [4552]:
df3.head()

,Temporada,#,Jugador,País,Posc,Equipo,Edad,Nacimiento,PJ,Titular,...,Ast90,G+A90,G-TP90,G+A-TP90,xG90,xAG90,xG+xAG90,npxG90,npxG+xAG90,Partidos
0,2017-2018,1,Vincent Aboubakar,cmCMR,DL,ptPorto,25,1992,6,6,...,0.36,1.27,0.91,1.27,0.51,0.19,0.70,0.51,0.70,Partidos
1,2017-2018,2,Marcos Acuña,arARG,"DL,DF",ptSporting CP,25,1991,5,5,...,0.21,0.21,0.00,0.21,0.00,0.11,0.11,0.00,0.11,Partidos
2,2017-2018,3,Tosin Adarabioyo,engENG,DF,engManchester City,19,1997,2,1,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Partidos
3,2017-2018,4,Adriano,brBRA,DF,trBeşiktaş,32,1984,6,5,...,0.00,0.00,0.00,0.00,0.01,0.02,0.03,0.01,0.03,Partidos
4,2017-2018,5,Luiz Adriano,brBRA,"DL,CC",ruSpartak Moscow,30,1987,6,6,...,0.00,0.17,0.17,0.17,0.16,0.15,0.32,0.16,0.32,Partidos


In [4553]:
#Como vemos tenemos columnas que se muestran como categoricas pero que en realidad son numericas, asi que vamos a convertirlas a numericas
columnas_numericas_df3=['Edad', 'Nacimiento', 'Titular', 'Mín', '90 s', 'Gls.', 'Ass', 'G+A',
       'G-TP', 'TP', 'TPint', 'TA', 'TR', 'xG', 'npxG', 'xAG', 'npxG+xAG',
       'PrgC', 'PrgP', 'PrgR', 'Gls90.', 'Ast90', 'G+A90', 'G-TP90',
       'G+A-TP90', 'xG90', 'xAG90', 'xG+xAG90', 'npxG90', 'npxG+xAG90']

# Convertir las columnas numéricas al tipo de datos apropiado
for columna in columnas_numericas_df3:
    df3[columna] = pd.to_numeric(df3[columna], errors='coerce')  # 'coerce' convierte los valores no numéricos a NaN

# Verificamos que las columnas se hayan convertido correctamente
print(df3.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5221 entries, 0 to 5220
Data columns (total 38 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Temporada   5221 non-null   object 
 1   #           5221 non-null   object 
 2   Jugador     5221 non-null   object 
 3   País        5216 non-null   object 
 4   Posc        5220 non-null   object 
 5   Equipo      5221 non-null   object 
 6   Edad        4298 non-null   float64
 7   Nacimiento  5022 non-null   float64
 8   PJ          5221 non-null   object 
 9   Titular     5023 non-null   float64
 10  Mín         4979 non-null   float64
 11  90 s        5023 non-null   float64
 12  Gls.        5023 non-null   float64
 13  Ass         5023 non-null   float64
 14  G+A         5023 non-null   float64
 15  G-TP        5023 non-null   float64
 16  TP          5023 non-null   float64
 17  TPint       5023 non-null   float64
 18  TA          5023 non-null   float64
 19  TR          5023 non-null  

Por ahora en este dataframe si mantrendremos todas las columnas, sin en¡mbargo observamos como en datasets anteriores que en nombre del equipo aparece el prefijo delante cosa que eliminaremos. Asi mismo el País aparece con prefijo cosa que también cambiaremos.

In [4554]:
#Quitamos los prefijos de los nombres de los equipos
#Anteriormente creamos una función para quitar los prefijos de los nombres de los equipos la cual usaremos ahora

# Aplicar la función para quitar prefijos en la columna 'Equipo'
df3['Equipo'] = df3['Equipo'].apply(lambda x: quitar_prefijos(x, prefijos))

# Verificar los cambios
df3.head()

,Temporada,#,Jugador,País,Posc,Equipo,Edad,Nacimiento,PJ,Titular,...,Ast90,G+A90,G-TP90,G+A-TP90,xG90,xAG90,xG+xAG90,npxG90,npxG+xAG90,Partidos
0,2017-2018,1,Vincent Aboubakar,cmCMR,DL,Porto,25.0,1992.0,6,6.0,...,0.36,1.27,0.91,1.27,0.51,0.19,0.70,0.51,0.70,Partidos
1,2017-2018,2,Marcos Acuña,arARG,"DL,DF",Sporting CP,25.0,1991.0,5,5.0,...,0.21,0.21,0.00,0.21,0.00,0.11,0.11,0.00,0.11,Partidos
2,2017-2018,3,Tosin Adarabioyo,engENG,DF,Manchester City,19.0,1997.0,2,1.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Partidos
3,2017-2018,4,Adriano,brBRA,DF,Beşiktaş,32.0,1984.0,6,5.0,...,0.00,0.00,0.00,0.00,0.01,0.02,0.03,0.01,0.03,Partidos
4,2017-2018,5,Luiz Adriano,brBRA,"DL,CC",Spartak Moscow,30.0,1987.0,6,6.0,...,0.00,0.17,0.17,0.17,0.16,0.15,0.32,0.16,0.32,Partidos


In [4555]:
# Diccionario de mapeo de abreviaciones de países a nombres completos

paises_nacimiento_dict = {
    'CMR': 'Camerún',
    'ESP': 'España',
    'ARG': 'Argentina',
    'ENG': 'Inglaterra',
    'BRA': 'Brasil',
    'RUS': 'Rusia',
    'TUR': 'Turquía',
    'POR': 'Portugal',
    'UKR': 'Ucrania',
    'GRE': 'Grecia',
    'CRO': 'Croacia',
    'NED': 'Países Bajos',
    'SUI': 'Suiza',
    'BEL': 'Bélgica',
    'AUT': 'Austria',
    'POL': 'Polonia',
    'SRB': 'Serbia',
    'SCO': 'Escocia',
    'CZE': 'República Checa',
    'SWE': 'Suecia',
    'NOR': 'Noruega',
    'DEN': 'Dinamarca',
    'BUL': 'Bulgaria',
    'ROU': 'Rumania',
    'HUN': 'Hungría',
    'SVK': 'Eslovaquia',
    'ISR': 'Israel',
    'FIN': 'Finlandia',
    'MLT': 'Malta',
    'LUX': 'Luxemburgo',
    'ISL': 'Islandia',
    'WAL': 'Gales',
    'IRL': 'Irlanda',
    'SCO': 'Escocia',
    'SCT': 'Escocia',
    'WAL': 'Gales',
    'CYP': 'Chipre',
    'LIE': 'Liechtenstein',
    'LVA': 'Letonia',
    'EST': 'Estonia',
    'LTU': 'Lituania',
    'AND': 'Andorra',
    'SMR': 'San Marino',
    'GIB': 'Gibraltar',
    'MLT': 'Malta',
    'MKD': 'Macedonia del Norte',
    'ALB': 'Albania',
    'MNE': 'Montenegro',
    'BIH': 'Bosnia y Herzegovina',
    'FRO': 'Islas Feroe',
    'ISL': 'Islandia',
    'SVN': 'Eslovenia',
    'MOL': 'Moldavia',
    'GEO': 'Georgia',
    'ARM': 'Armenia',
    'AZE': 'Azerbaiyán',
    'KAZ': 'Kazajistán',
    'FRA': 'Francia',
    'GER': 'Alemania',
    'ITA': 'Italia',
    'SUI': 'Suiza',
    'SWE': 'Suecia',
    'NOR': 'Noruega',
    'DEN': 'Dinamarca',
    'FIN': 'Finlandia',
    'POL': 'Polonia',
    'CZE': 'República Checa',
    'SVK': 'Eslovaquia',
    'HUN': 'Hungría',
    'ROU': 'Rumania',
    'MAR': 'Marruecos',
    'URU': 'Uruguay',
    'CIV': 'Costa de Marfil',
    'ALG': 'Argelia',
    'CHI': 'Chile',
    'COL': 'Colombia',
    'MEX': 'México',
    'KAN': 'Congo',
    'KOR': 'Corea del Sur',
    'SUR': 'Surinam',
    'CAN': 'Canadá',
    'TUN': 'Túnez',
    'HAI': 'Haití',
    'CRC': 'Costa Rica',
    'AUS': 'Australia',
    'JPN': 'Japón',
    'GUI': 'Guinea',
    'SEN': 'Senegal',
    'CPV': 'Cabo Verde',
    'MLI': 'Malí',
    'PAR': 'Paraguay',
    'NGA': 'Nigeria',
    'COM': 'Comoras',
    'HON': 'Honduras',
    'RSA': 'Sudáfrica',
    'GHA': 'Ghana',
    'BEN': 'Benín',
    'TOG': 'Togo',
    'GAM': 'Gambia',
    'EGY': 'Egipto',
    'ECU': 'Ecuador',
    'MAS': 'Malasia',
    'NZL': 'Nueva Zelanda',
    'THA': 'Tailandia',
    'IRN': 'Irán',
    'IRQ': 'Irak',
    'SYR': 'Siria',
    'JOR': 'Jordania',
    'KSA': 'Arabia Saudita',
    'USA': 'Estados Unidos',
    'CTA': 'Catar',
    'DOM': 'República Dominicana',
    'MAD': 'Madagascar',
    'ZIM': 'Zimbabue',
    'BFA': 'Burkina',
    'KEN': 'Kenia',
    'JAM': 'Jamaica',
    'COD': 'República Democrática del Congo',
    'ZAM': 'Zambia',
    'KGZ': 'Kirgu',
    'GAB': 'Gabón',
    'MOZ': 'Mozambique',
    'NAM': 'Namibia',
    'UGA': 'Uganda',
    'BOT': 'Botsuana',
    'GUI': 'Guinea',
    'SUD': 'Sudán',
    'SLE': 'Sierra Leona',
    'CHA': 'Chad',
    'TAN': 'Tanzania',
    'ANG': 'Angola',
    'VEN': 'Venezuela',
    'GNB': 'Guinea-Bisáu',
    'UZB': 'Uzbekistán',


}

df3['País'] = df3['País'].str[-3:]  # Suponiendo que las abreviaciones están al final de la cadena

# Reemplazar las abreviaciones con los nombres completos de los países
df3['País'] = df3['País'].replace(paises_nacimiento_dict)

# Verificar los cambios
df3.head()


,Temporada,#,Jugador,País,Posc,Equipo,Edad,Nacimiento,PJ,Titular,...,Ast90,G+A90,G-TP90,G+A-TP90,xG90,xAG90,xG+xAG90,npxG90,npxG+xAG90,Partidos
0,2017-2018,1,Vincent Aboubakar,Camerún,DL,Porto,25.0,1992.0,6,6.0,...,0.36,1.27,0.91,1.27,0.51,0.19,0.70,0.51,0.70,Partidos
1,2017-2018,2,Marcos Acuña,Argentina,"DL,DF",Sporting CP,25.0,1991.0,5,5.0,...,0.21,0.21,0.00,0.21,0.00,0.11,0.11,0.00,0.11,Partidos
2,2017-2018,3,Tosin Adarabioyo,Inglaterra,DF,Manchester City,19.0,1997.0,2,1.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Partidos
3,2017-2018,4,Adriano,Brasil,DF,Beşiktaş,32.0,1984.0,6,5.0,...,0.00,0.00,0.00,0.00,0.01,0.02,0.03,0.01,0.03,Partidos
4,2017-2018,5,Luiz Adriano,Brasil,"DL,CC",Spartak Moscow,30.0,1987.0,6,6.0,...,0.00,0.17,0.17,0.17,0.16,0.15,0.32,0.16,0.32,Partidos


Vemos que la columna 'Partidos' no nos ofrece ninguna información relevante, por lo que pasamos a borrarla.

In [4556]:
df3.drop(["Partidos"],axis=1,inplace=True)

In [4557]:
#Comprobamos que se haya eliminado correctamente
df3.columns

Index(['Temporada', '#', 'Jugador', 'País', 'Posc', 'Equipo', 'Edad',
       'Nacimiento', 'PJ', 'Titular', 'Mín', '90 s', 'Gls.', 'Ass', 'G+A',
       'G-TP', 'TP', 'TPint', 'TA', 'TR', 'xG', 'npxG', 'xAG', 'npxG+xAG',
       'PrgC', 'PrgP', 'PrgR', 'Gls90.', 'Ast90', 'G+A90', 'G-TP90',
       'G+A-TP90', 'xG90', 'xAG90', 'xG+xAG90', 'npxG90', 'npxG+xAG90'],
      dtype='object')

Analizando el CSV nos dimos también cuenta que se repitía el "encabezado" por lo que vamos a borrar estas filas.

In [4558]:
# Definir la fila de comparación con 'Temporada' en blanco
fila_comparacion = [''] + ['RL', 'Jugador', 'aís', 'Posc', 'Equipo', 'Edad', 'Nacimiento', 'PJ', 'Titular', 'Mín', '90 s', 'Gls.', 'Ass', 'G+A', 'G-TP', 'TP', 'TPint', 'TA', 'TR', 'xG', 'npxG', 'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'PrgR', 'Gls.', 'Ast', 'G+A', 'G-TP', 'G+A-TP', 'xG', 'xAG', 'xG+xAG', 'npxG', 'npxG+xAG', 'Partidos']

# Inicializamos una lista para almacenar las filas que queremos mantener
filas_mantener = []

# Iteramos sobre las filas del DataFrame df3
for index, row in df3.iterrows():
    # Comparamos los valores de las columnas excepto 'Temporada'
    valores_iguales = True
    for valor_fila, valor_comparacion in zip(row.values[1:], fila_comparacion[1:]):
        if valor_fila != valor_comparacion:
            valores_iguales = False
            break
    # Si no son iguales, añadimos la fila a las filas a mantener
    if not valores_iguales:
        filas_mantener.append(row)

# Creamos un nuevo DataFrame con las filas que queremos mantener
df3= pd.DataFrame(filas_mantener)

In [4561]:
# Eliminar las filas que contienen solo comas consecutivas
df3_filtrado = df3.dropna(thresh=len(df3.columns) - 5)

# Verificar el DataFrame resultante
df3_filtrado.shape




(5019, 37)

In [4560]:
#lo pasamos a un csv limpio
df3_filtrado.to_csv('../data/jugadores_limpio.csv', index=False)